# Sales Tax Inflation Adjustment

1. Adjusts all the years after 2019 for inflation with 2019 as the baseline
2. Brings data into graphing format
    - If you just want to graph, jump to the graphing section and run.

In [49]:
import pandas as pd
import cpi
from datetime import date, datetime, timedelta
import requests
import time
from io import StringIO


#pd.set_option('display.max_rows', 1000) 
#pd.set_option('display.max_columns', 1000)
#pd.set_option("display.max_colwidth", None) 

In [45]:
cpi.update()

/Users/anushasubramanian/.pyenv/versions/3.12.7/lib/python3.12/site-packages/cpi/download.py:175: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(response.text), sep="\t")
/Users/anushasubramanian/.pyenv/versions/3.12.7/lib/python3.12/site-packages/cpi/download.py:175: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(response.text), sep="\t")
/Users/anushasubramanian/.pyenv/versions/3.12.7/lib/python3.12/site-packages/cpi/download.py:175: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(response.text), sep="\t")
/Users/anushasubramanian/.pyenv/versions/3.12.7/lib/python3.12/site-packages/cpi/download.py:175: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringI

In [51]:
url = "https://data.sfgov.org/resource/w6qv-daq5.csv"
all_data = []
offset = 0
limit = 1000  

expected_columns = None

filters = {
    '$limit': limit,
    '$offset': offset,
}

while True:
    # Update offset for pagination
    filters['$offset'] = offset
    
    # Make request
    try:
        response = requests.get(url, params=filters, timeout=30)
    except Exception as e:
        print(f"Request failed: {e}")
        break
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break
    
    # parse CSV
    chunk_df = pd.read_csv(StringIO(response.text))
    
    if offset == 0:
        expected_columns = chunk_df.columns
    
    # check if we got any data. If we got nothing, we've reached the end of the list
    if len(chunk_df) == 0:
        print(f"No more data at offset {offset}")
        break
    
    # creating a list of dfs to concat at the end
    all_data.append(chunk_df)
    
    # if we got fewer records than the limit, we've reached the end
    if len(chunk_df) < limit:
        print(f"Last chunk (got {len(chunk_df)} < {limit} records)")
        break
    
    # update offset and wait so we don't timeout accidentally
    offset += limit
    time.sleep(0.5)
    
# concat
if all_data:
    sales_tax = pd.concat(all_data, ignore_index=True)
elif expected_columns is not None:
    sales_tax = pd.DataFrame(columns=expected_columns)
else:
    sales_tax = pd.DataFrame()
    
if len(sales_tax) == 0:
    print(f"No new data retrieved")
    
else:
    print(f"Total records fetched: {len(sales_tax)}")

    sales_tax.head()

Last chunk (got 376 < 1000 records)
Total records fetched: 1376


In [52]:
sales_tax.head()

,quarter,quarter_start_date,area_type,area,amount,data_as_of,data_loaded_at
0,2025Q3,2025-07-01T00:00:00.000,Neighborhood,Bayview Hunters Point,2665197.19,2026-01-26T15:53:05.000,2026-01-26T16:40:08.396
1,2025Q3,2025-07-01T00:00:00.000,Neighborhood,Bernal Heights,448018.80,2026-01-26T15:53:05.000,2026-01-26T16:40:08.396
2,2025Q3,2025-07-01T00:00:00.000,Neighborhood,Castro-Upper Market,571049.04,2026-01-26T15:53:05.000,2026-01-26T16:40:08.396
3,2025Q3,2025-07-01T00:00:00.000,Neighborhood,Chinatown,1072661.93,2026-01-26T15:53:05.000,2026-01-26T16:40:08.396
4,2025Q3,2025-07-01T00:00:00.000,Neighborhood,Excelsior,173582.37,2026-01-26T15:53:05.000,2026-01-26T16:40:08.396


In [53]:
sales_tax["area"].value_counts()

area
Bayview Hunters Point         43
Bernal Heights                43
West of Twin Peaks            43
Visitacion Valley             43
Tenderloin                    43
Sunset-Parkside               43
South of Market               43
Russian Hill                  43
Presidio Heights              43
Potrero Hill                  43
Portola                       43
Pacific Heights               43
Outer Richmond                43
Oceanview-Merced-Ingleside    43
North Beach                   43
Noe Valley                    43
Nob Hill                      43
Mission Bay                   43
Mission                       43
Marina                        43
Lone Mountain-USF             43
Lakeshore                     43
Japantown                     43
Inner Richmond                43
Hayes Valley                  43
Haight Ashbury                43
Golden Gate Park              43
Glen Park                     43
Excelsior                     43
Chinatown                     43
Castr

In [54]:
neighborhoods = ["Tenderloin","Inner Richmond", "Outer Richmond", "Mission", "Excelsior", "Bayview Hunters Point", "Sunset-Parkside"]

In [58]:
sales_tax = sales_tax[sales_tax["area"].isin(neighborhoods)]

In [73]:
sales_tax = sales_tax.pivot_table(index = "area",
                     columns = "quarter",
                     values = "amount",
                     aggfunc = "sum").reset_index()

In [74]:
sales_tax.columns = sales_tax.columns.str.strip()

In [85]:
# columns we want to retain
quarter_columns = [column for column in sales_tax.columns if column.startswith(('2019', '2020', '2021', '2022', '2023', '2024', '2025'))]

In [87]:
sales_tax = sales_tax[['area'] + quarter_columns]
sales_tax.head()

quarter,area,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,...,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1,2025Q2,2025Q3
0,Bayview Hunters Point,2949031.40,3397922.89,3603746.60,3238831.70,2241349.19,2740673.05,2996372.25,2427499.44,2352774.84,...,3403331.12,3029022.22,2864026.23,2745924.81,2844217.82,2716935.92,2595644.36,2478447.38,2708224.49,2665197.19
1,Excelsior,162835.22,173095.90,172197.92,171801.77,87953.81,153933.73,149643.62,132061.91,120643.99,...,171111.24,179666.64,169838.76,180944.16,176310.07,161839.91,188303.48,169124.62,191093.01,173582.37
2,Inner Richmond,630450.66,691382.42,711369.32,695014.63,428198.37,586447.57,571489.97,565201.38,588913.38,...,747266.48,566469.16,822087.23,744301.13,736352.73,732650.39,824655.79,738803.23,824032.52,775941.83
3,Mission,3551567.39,3731289.83,3923585.21,4010635.39,2090562.46,2587014.51,2835961.27,2474197.20,2670713.91,...,3559128.70,3625164.93,4076960.77,3150897.14,3440265.90,3477672.92,3544461.01,3679348.86,3616643.17,3582094.16
4,Outer Richmond,534103.93,558399.57,567121.06,596952.85,245585.50,417351.96,367783.31,321298.85,322539.45,...,506680.07,486359.39,477861.17,481623.74,487395.57,462539.85,481224.57,470583.98,465639.07,489745.73


In [89]:
quarter_to_month = {1: 3, 2: 6, 3: 9, 4: 12}

for column in quarter_columns:
    year = int(column[:4]) # output will be like "2018" 
    quarter = int(column[5]) # output will be like "2" 
    month = quarter_to_month[quarter] 
    
    # Create new column with 'adj_' prefix
    # Adjust to the SAME quarter but in 2019 
    sales_tax[f'adj_{column}'] = sales_tax[column].apply(lambda x: cpi.inflate(x, date(year, month, 28), to=date(2019, month, 28)))

/var/folders/2_/nl4h85lj1jqcl64vgwgvqnh40000gn/T/ipykernel_13769/3493739873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_tax[f'adj_{column}'] = sales_tax[column].apply(lambda x: cpi.inflate(x, date(year, month, 28), to=date(2019, month, 28)))
/var/folders/2_/nl4h85lj1jqcl64vgwgvqnh40000gn/T/ipykernel_13769/3493739873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_tax[f'adj_{column}'] = sales_tax[column].apply(lambda x: cpi.inflate(x, date(year, month, 28), to=date(2019, month, 28))

In [90]:
sales_tax_adj = sales_tax

In [98]:
sales_tax_adj = sales_tax[["area"] +
                           [column for column in sales_tax.columns if column.startswith('adj')]]

In [99]:
sales_tax_adj.head()

quarter,area,adj_2019Q1,adj_2019Q2,adj_2019Q3,adj_2019Q4,adj_2020Q1,adj_2020Q2,adj_2020Q3,adj_2020Q4,adj_2021Q1,...,adj_2023Q2,adj_2023Q3,adj_2023Q4,adj_2024Q1,adj_2024Q2,adj_2024Q3,adj_2024Q4,adj_2025Q1,adj_2025Q2,adj_2025Q3
0,Bayview Hunters Point,2949031.40,3397922.89,3603746.60,3238831.70,2.207371e+06,2.723089e+06,2.955838e+06,2.394881e+06,2.257954e+06,...,2.857141e+06,2.526824e+06,2.399315e+06,2.234864e+06,2.318856e+06,2.212482e+06,2.113443e+06,1.970070e+06,2.150578e+06,2.106876e+06
1,Excelsior,162835.22,173095.90,172197.92,171801.77,8.662044e+04,1.529461e+05,1.476193e+05,1.302874e+05,1.157818e+05,...,1.436501e+05,1.498787e+05,1.422811e+05,1.472675e+05,1.437434e+05,1.317911e+05,1.533217e+05,1.344339e+05,1.517454e+05,1.372193e+05
2,Inner Richmond,630450.66,691382.42,711369.32,695014.63,4.217069e+05,5.826850e+05,5.637590e+05,5.576067e+05,5.651792e+05,...,6.273400e+05,4.725512e+05,6.886970e+05,6.057747e+05,6.003393e+05,5.966190e+05,6.714567e+05,5.872603e+05,6.543574e+05,6.133930e+05
3,Mission,3551567.39,3731289.83,3923585.21,4010635.39,2.058870e+06,2.570416e+06,2.797597e+06,2.440951e+06,2.563080e+06,...,2.987935e+06,3.024129e+06,3.415441e+06,2.564465e+06,2.804806e+06,2.831973e+06,2.885995e+06,2.924643e+06,2.871946e+06,2.831696e+06
4,Outer Richmond,534103.93,558399.57,567121.06,596952.85,2.418624e+05,4.146743e+05,3.628080e+05,3.169815e+05,3.095406e+05,...,4.253646e+05,4.057232e+05,4.003244e+05,3.919858e+05,3.973676e+05,3.766600e+05,3.918259e+05,3.740580e+05,3.697601e+05,3.871509e+05


In [102]:
sales_tax_adj.to_csv("adjusted_sales_tax.csv", index = False)

## Graph formatting

In [103]:
sales_tax_adj = pd.read_csv("adjusted_sales_tax.csv")
sales_tax_adj.head()

,area,adj_2019Q1,adj_2019Q2,adj_2019Q3,adj_2019Q4,adj_2020Q1,adj_2020Q2,adj_2020Q3,adj_2020Q4,adj_2021Q1,...,adj_2023Q2,adj_2023Q3,adj_2023Q4,adj_2024Q1,adj_2024Q2,adj_2024Q3,adj_2024Q4,adj_2025Q1,adj_2025Q2,adj_2025Q3
0,Bayview Hunters Point,2949031.40,3397922.89,3603746.60,3238831.70,2.207371e+06,2.723089e+06,2.955838e+06,2.394881e+06,2.257954e+06,...,2.857141e+06,2.526824e+06,2.399315e+06,2.234864e+06,2.318856e+06,2.212482e+06,2.113443e+06,1.970070e+06,2.150578e+06,2.106876e+06
1,Excelsior,162835.22,173095.90,172197.92,171801.77,8.662044e+04,1.529461e+05,1.476193e+05,1.302874e+05,1.157818e+05,...,1.436501e+05,1.498787e+05,1.422811e+05,1.472675e+05,1.437434e+05,1.317911e+05,1.533217e+05,1.344339e+05,1.517454e+05,1.372193e+05
2,Inner Richmond,630450.66,691382.42,711369.32,695014.63,4.217069e+05,5.826850e+05,5.637590e+05,5.576067e+05,5.651792e+05,...,6.273400e+05,4.725512e+05,6.886970e+05,6.057747e+05,6.003393e+05,5.966190e+05,6.714567e+05,5.872603e+05,6.543574e+05,6.133930e+05
3,Mission,3551567.39,3731289.83,3923585.21,4010635.39,2.058870e+06,2.570416e+06,2.797597e+06,2.440951e+06,2.563080e+06,...,2.987935e+06,3.024129e+06,3.415441e+06,2.564465e+06,2.804806e+06,2.831973e+06,2.885995e+06,2.924643e+06,2.871946e+06,2.831696e+06
4,Outer Richmond,534103.93,558399.57,567121.06,596952.85,2.418624e+05,4.146743e+05,3.628080e+05,3.169815e+05,3.095406e+05,...,4.253646e+05,4.057232e+05,4.003244e+05,3.919858e+05,3.973676e+05,3.766600e+05,3.918259e+05,3.740580e+05,3.697601e+05,3.871509e+05


In [104]:
sales_tax_adj.columns

Index(['area', 'adj_2019Q1', 'adj_2019Q2', 'adj_2019Q3', 'adj_2019Q4',
       'adj_2020Q1', 'adj_2020Q2', 'adj_2020Q3', 'adj_2020Q4', 'adj_2021Q1',
       'adj_2021Q2', 'adj_2021Q3', 'adj_2021Q4', 'adj_2022Q1', 'adj_2022Q2',
       'adj_2022Q3', 'adj_2022Q4', 'adj_2023Q1', 'adj_2023Q2', 'adj_2023Q3',
       'adj_2023Q4', 'adj_2024Q1', 'adj_2024Q2', 'adj_2024Q3', 'adj_2024Q4',
       'adj_2025Q1', 'adj_2025Q2', 'adj_2025Q3'],
      dtype='object')

In [105]:
# rename for graph

sales_tax_adj.columns = [c.replace("adj_", "").replace("Q", " Q") for c in sales_tax_adj.columns]
sales_tax_adj

,area,2019 Q1,2019 Q2,2019 Q3,2019 Q4,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,...,2023 Q2,2023 Q3,2023 Q4,2024 Q1,2024 Q2,2024 Q3,2024 Q4,2025 Q1,2025 Q2,2025 Q3
0,Bayview Hunters Point,2949031.40,3397922.89,3603746.60,3238831.70,2.207371e+06,2.723089e+06,2.955838e+06,2.394881e+06,2.257954e+06,...,2.857141e+06,2.526824e+06,2.399315e+06,2.234864e+06,2.318856e+06,2.212482e+06,2.113443e+06,1.970070e+06,2.150578e+06,2.106876e+06
1,Excelsior,162835.22,173095.90,172197.92,171801.77,8.662044e+04,1.529461e+05,1.476193e+05,1.302874e+05,1.157818e+05,...,1.436501e+05,1.498787e+05,1.422811e+05,1.472675e+05,1.437434e+05,1.317911e+05,1.533217e+05,1.344339e+05,1.517454e+05,1.372193e+05
2,Inner Richmond,630450.66,691382.42,711369.32,695014.63,4.217069e+05,5.826850e+05,5.637590e+05,5.576067e+05,5.651792e+05,...,6.273400e+05,4.725512e+05,6.886970e+05,6.057747e+05,6.003393e+05,5.966190e+05,6.714567e+05,5.872603e+05,6.543574e+05,6.133930e+05
3,Mission,3551567.39,3731289.83,3923585.21,4010635.39,2.058870e+06,2.570416e+06,2.797597e+06,2.440951e+06,2.563080e+06,...,2.987935e+06,3.024129e+06,3.415441e+06,2.564465e+06,2.804806e+06,2.831973e+06,2.885995e+06,2.924643e+06,2.871946e+06,2.831696e+06
4,Outer Richmond,534103.93,558399.57,567121.06,596952.85,2.418624e+05,4.146743e+05,3.628080e+05,3.169815e+05,3.095406e+05,...,4.253646e+05,4.057232e+05,4.003244e+05,3.919858e+05,3.973676e+05,3.766600e+05,3.918259e+05,3.740580e+05,3.697601e+05,3.871509e+05
5,Sunset-Parkside,823591.72,766165.91,743125.87,792780.84,3.907975e+05,6.943154e+05,6.541208e+05,5.770916e+05,5.387613e+05,...,7.140100e+05,7.074895e+05,6.649482e+05,7.112549e+05,7.030066e+05,7.077597e+05,6.981485e+05,6.607422e+05,6.917190e+05,6.927275e+05
6,Tenderloin,2129438.95,2655256.68,2120191.41,1562631.89,1.108613e+06,7.562480e+05,5.862045e+05,4.487793e+05,5.610142e+05,...,1.454866e+06,9.843074e+05,9.266369e+05,9.553353e+05,1.994373e+06,8.459500e+05,8.054206e+05,1.708463e+06,1.237094e+06,8.765749e+05


In [106]:
# transpose
sales_tax_adj = sales_tax_adj.T

# use first row as column headers (neighborhood names)
sales_tax_adj.columns = sales_tax_adj.loc["area"]

# format - drop that header row
sales_tax_adj = sales_tax_adj.drop(index="area")

# take out pandas column label clutter
sales_tax_adj.columns.name = None
sales_tax_adj.index.name = "quarter"
sales_tax_adj = sales_tax_adj.reset_index()

sales_tax_adj.head()

,quarter,Bayview Hunters Point,Excelsior,Inner Richmond,Mission,Outer Richmond,Sunset-Parkside,Tenderloin
0,2019 Q1,2949031.4,162835.22,630450.66,3551567.39,534103.93,823591.72,2129438.95
1,2019 Q2,3397922.89,173095.9,691382.42,3731289.83,558399.57,766165.91,2655256.68
2,2019 Q3,3603746.6,172197.92,711369.32,3923585.21,567121.06,743125.87,2120191.41
3,2019 Q4,3238831.7,171801.77,695014.63,4010635.39,596952.85,792780.84,1562631.89
4,2020 Q1,2207370.539474,86620.438214,421706.92153,2058869.722631,241862.446084,390797.528165,1108613.375081


In [107]:
sales_tax_adj.to_csv("adj_sales_tax_graph.csv", index = False)